In [1]:
%load_ext autoreload
%load_ext autotime

time: 394 µs (started: 2023-03-16 14:13:18 -07:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 1.29 s (started: 2023-03-16 14:13:18 -07:00)


In [3]:
import dask.dataframe as dd
import numpy as np
import pandas as pd


time: 555 ms (started: 2023-03-16 14:13:20 -07:00)


In [4]:
%autoreload 2

from wisio.recorder import RecorderAnalyzer

log_dir = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
        force_local=True,
        cluster_settings=dict(
            cores=4,
            dashboard_port=3666,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

time: 2.14 s (started: 2023-03-16 14:13:20 -07:00)


In [5]:
[client.dashboard_link for client in recorder_analyzer.cluster_manager.clients]

['http://192.168.135.144:3666/status']

time: 2.96 ms (started: 2023-03-16 14:13:23 -07:00)


In [6]:
%autoreload 2

res = recorder_analyzer.analyze_parquet(log_dir=log_dir)

time: 5.05 s (started: 2023-03-16 14:13:32 -07:00)


In [7]:
views, bottlenecks = res

time: 839 µs (started: 2023-03-16 14:13:41 -07:00)


In [11]:
from wisio.rules import Rule
from wisio._recorder.rules import RecorderRuleEngine

rules = dict(
    file_name=[Rule.METADATA_ACCESS_ISSUE, Rule.SMALL_IO_ACCESS],
    proc_name=[Rule.METADATA_ACCESS_ISSUE],
    trange=[],
)

rule_engine = RecorderRuleEngine(rules=rules)

rule_engine.process_bottlenecks(bottlenecks=bottlenecks)


((<Rule.METADATA_ACCESS_ISSUE: 21>, ('file_name',), {}),
 (<Rule.SMALL_IO_ACCESS: 30>,
  ('file_name',),
  {'/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc3372.fits': RuleResult(rule=<Rule.SMALL_IO_ACCESS: 30>, description="'/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc3372.fits' is accessed by process(es) 'mAddMPI#lassen246#41731#35184372406720' during the 1th second(s) with a transfer size smaller than 256KB (100.27KB).", reasons=[RuleReason(description="'write' time is 99.66% (4.50 seconds) of I/O time.", value=0.9966153583826323)]),
   '/p/gpfs1/iopp/temp/montage/2854964/25/run/ngc3372_area.fits': RuleResult(rule=<Rule.SMALL_IO_ACCESS: 30>, description="'/p/gpfs1/iopp/temp/montage/2854964/25/run/ngc3372_area.fits' is accessed by process(es) 'mAddMPI#lassen578#12579#35184372406720' during the 1th second(s) with a transfer size smaller than 256KB (20.82KB).", reasons=[RuleReason(description="'write' time is 99.93% (4.89 seconds) of I/O time.", value=0.9992767731217669)])}),
 (<Rule.META

time: 2.42 s (started: 2023-03-16 14:19:45 -07:00)


In [11]:
for view_key, bottleneck in bottlenecks.items():
    for th in bottleneck.keys():
        print(view_key, th, len(bottleneck[th]))

('trange',) 0.001 42
('trange',) 0.010 14
('trange',) 0.100 3
('trange',) 0.250 1
('trange',) 0.500 0
('trange',) 0.750 0
('file_name',) 0.001 725
('file_name',) 0.010 113
('file_name',) 0.100 10
('file_name',) 0.250 4
('file_name',) 0.500 0
('file_name',) 0.750 0
('proc_name',) 0.001 74
('proc_name',) 0.010 61
('proc_name',) 0.100 6
('proc_name',) 0.250 3
('proc_name',) 0.500 0
('proc_name',) 0.750 0
('trange', 'file_name') 0.001 725
('trange', 'file_name') 0.010 113
('trange', 'file_name') 0.100 10
('trange', 'file_name') 0.250 4
('trange', 'file_name') 0.500 0
('trange', 'file_name') 0.750 0
('trange', 'proc_name') 0.001 74
('trange', 'proc_name') 0.010 61
('trange', 'proc_name') 0.100 6
('trange', 'proc_name') 0.250 3
('trange', 'proc_name') 0.500 0
('trange', 'proc_name') 0.750 0
('file_name', 'trange') 0.001 42
('file_name', 'trange') 0.010 14
('file_name', 'trange') 0.100 3
('file_name', 'trange') 0.250 1
('file_name', 'trange') 0.500 0
('file_name', 'trange') 0.750 0
('file_nam

In [8]:
!rm -rf {log_dir}/bottlenecks

time: 343 ms (started: 2023-03-12 17:52:39 -07:00)


In [9]:
recorder_analyzer.save_bottlenecks(log_dir=log_dir, bottlenecks=bottlenecks)


time: 7.35 s (started: 2023-03-12 17:52:39 -07:00)


In [10]:
!chmod g+r {log_dir}/bottlenecks/*
!ls -lh {log_dir}/bottlenecks

total 27M
-rw-r--r-- 1 izzet iopp 2.5M Mar 12 17:52 file_name.json
-rw-r--r-- 1 izzet iopp 1.5M Mar 12 17:52 file_name_proc_name.json
-rw-r--r-- 1 izzet iopp 1.4M Mar 12 17:52 file_name_proc_name_trange.json
-rw-r--r-- 1 izzet iopp 1.4M Mar 12 17:52 file_name_trange.json
-rw-r--r-- 1 izzet iopp 1.5M Mar 12 17:52 file_name_trange_proc_name.json
-rw-r--r-- 1 izzet iopp 1.5M Mar 12 17:52 proc_name.json
-rw-r--r-- 1 izzet iopp 2.5M Mar 12 17:52 proc_name_file_name.json
-rw-r--r-- 1 izzet iopp 1.4M Mar 12 17:52 proc_name_file_name_trange.json
-rw-r--r-- 1 izzet iopp 1.4M Mar 12 17:52 proc_name_trange.json
-rw-r--r-- 1 izzet iopp 2.5M Mar 12 17:52 proc_name_trange_file_name.json
-rw-r--r-- 1 izzet iopp 1.4M Mar 12 17:52 trange.json
-rw-r--r-- 1 izzet iopp 2.5M Mar 12 17:52 trange_file_name.json
-rw-r--r-- 1 izzet iopp 1.5M Mar 12 17:52 trange_file_name_proc_name.json
-rw-r--r-- 1 izzet iopp 1.5M Mar 12 17:52 trange_proc_name.json
-rw-r--r-- 1 izzet iopp 2.5M Mar 12 17:52 trange_proc_name_fil